In [2]:
import uproot
import matplotlib.pyplot as plt
import mplhep as hep
import numpy as np
import pandas as pd
import awkward as ak
import vector

In [3]:
def Read_Tree(path):
    tree = uproot.open(path_sm)['LHEF']
    Evt_num = tree.num_entries
    print('there are {0} numebr of events'.format(Evt_num))
    Particles = tree.arrays(filter_name='Particle*')
    
    is_Ele = abs(Particles['Particle.PID']) == 11
    is_Ele_ch = ak.num(Particles['Particle.PID'][is_Ele]) == 2
    
    is_Mu = abs(Particles['Particle.PID']) == 13
    is_Mu_ch = ak.num(Particles['Particle.PID'][is_Mu]) == 2
    
    Particles_ele_ch = Particles[is_Ele_ch ]
    Particles_mu_ch  = Particles[is_Mu_ch ]
    
    
    return Evt_num,Particles_ele_ch,Particles_mu_ch

In [4]:
def Make_Object_Array(Particles):    
    Photon_mask   = (Particles['Particle.PID'] == 22)
    Photon = ak.zip({
    "PT" :    Particles['Particle.PT'][Photon_mask],
    "Eta":    Particles['Particle.Eta'][Photon_mask],
    "Phi":    Particles['Particle.Phi'][Photon_mask],
    "E" :    Particles['Particle.E'][Photon_mask],
    "Px" : Particles['Particle.Px'][Photon_mask],
    "Py" : Particles['Particle.Py'][Photon_mask],  
    "Pz" : Particles['Particle.Pz'][Photon_mask],  
    })
    
    Electron_mask = abs(Particles['Particle.PID']) == 11
    Electron = ak.zip({
    "PT" :    Particles['Particle.PT'][Electron_mask],
    "Eta":    Particles['Particle.Eta'][Electron_mask],
    "Phi":    Particles['Particle.Phi'][Electron_mask],
    "E" :    Particles['Particle.E'][Electron_mask],
    "Px" : Particles['Particle.Px'][Electron_mask],
    "Py" : Particles['Particle.Py'][Electron_mask],  
    "Pz" : Particles['Particle.Pz'][Electron_mask],  
    })   
    
    Muon_mask = abs(Particles['Particle.PID']) == 13
    Muon = ak.zip({
    "PT" :    Particles['Particle.PT'][Muon_mask],
    "Eta":    Particles['Particle.Eta'][Muon_mask],
    "Phi":    Particles['Particle.Phi'][Muon_mask],
    "E" :    Particles['Particle.E'][Muon_mask],
    "Px" : Particles['Particle.Px'][Muon_mask],
    "Py" : Particles['Particle.Py'][Muon_mask],  
    "Pz" : Particles['Particle.Pz'][Muon_mask],  
    })
    
    W_mask = abs(Particles['Particle.PID']) ==24 
    Wboson = ak.zip({
    "PT" :    Particles['Particle.PT'][W_mask],
    "Eta":    Particles['Particle.Eta'][W_mask],
    "Phi":    Particles['Particle.Phi'][W_mask],
    "E" :    Particles['Particle.E'][W_mask],
    "Px" : Particles['Particle.Px'][W_mask],
    "Py" : Particles['Particle.Py'][W_mask],  
    "Pz" : Particles['Particle.Pz'][W_mask],  
    })

    
    return Electron,Muon,Photon,Wboson

In [10]:
def Analysis(chmark,Particles):
    
    print("Start analysis {0} channel".format(chmark))
    Electron,Muon,Photon,Wboson = Make_Object_Array(Particles)
    
    if chmark=='Electron':
        print("test -- contain muon? ",ak.sum(ak.num(Muon)))
        print("test -- does not have diele? ",ak.sum(ak.num(Electron) !=2))
        Lep1_vec = vector.obj(px=Electron[:,0].Px,py=Electron[:,0].Py,pz=Electron[:,0].Pz,E=Electron[:,0].E)
        Lep2_vec = vector.obj(px=Electron[:,1].Px,py=Electron[:,1].Py,pz=Electron[:,1].Pz,E=Electron[:,1].E)
        
    elif chmark=='Muon':
        print("test -- contain Electron? ",ak.sum(ak.num(Electron)))
        print("test -- does not have dimu? ",ak.sum(ak.num(Muon) !=2))
        Lep1_vec = vector.obj(px=Muon[:,0].Px,py=Muon[:,0].Py,pz=Muon[:,0].Pz,E=Muon[:,0].E)
        Lep2_vec = vector.obj(px=Muon[:,1].Px,py=Muon[:,1].Py,pz=Muon[:,1].Pz,E=Muon[:,1].E)
    else:
        raise NameError('unavailable channel name')

    W_vec    = vector.obj(px=Wboson[:,0].Px,py=Wboson[:,0].Py,pz=Wboson[:,0].Pz,E=Wboson[:,0].E)
    WZ_vec    = Lep1_vec + Lep2_vec + W_vec    
    
    Mwz   =  WZ_vec.M.to_numpy()
    phoPT =  ak.flatten(Photon.PT).to_numpy()
    return Mwz,phoPT 

In [21]:
def Process(path):
    Evt_num,Particles_ele_ch,Particles_mu_ch = Read_Tree(path_sm)
    
    print("Elech + Much == Evtnum ? ",len(Particles_ele_ch) + len(Particles_mu_ch) == Evt_num)
    ele_Mwz,ele_phoPT   = Analysis("Electron",Particles_ele_ch)
    mu_Mwz,mu_phoPT   = Analysis("Muon",Particles_mu_ch)
    Mwz_arr   = np.concatenate((ele_Mwz, mu_Mwz), axis = 0)
    phoPT_arr = np.concatenate((ele_phoPT, mu_phoPT), axis = 0)
    
    return Mwz_arr,phoPT_arr

In [23]:
path_sm            = "SM_WZG/unweighted_events.root"
path_aQGC_allzeros = "aQGC_Allzeros/unweighted_events.root"
path_aQGC_FM       = "aQGC_FM/unweighted_events.root"
path_aQGC_FT       = "aQGC_FT/unweighted_events.root"

In [25]:
pd.DataFrame('sm',"SM_WZG/unweighted_events.root",)

""
